We use word embeddings to calculate a relevence score for each tweet. This quantifies how relevant each tweet is to the main themes.

In [3]:
pip install flair

     |████████████████████████████████| 322 kB 5.5 MB/s 
     |████████████████████████████████| 64 kB 2.3 MB/s 
     |████████████████████████████████| 19.7 MB 4.2 MB/s 
     |████████████████████████████████| 48 kB 5.0 MB/s 
     |████████████████████████████████| 788 kB 56.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 3.5 MB 42.6 MB/s 
     |████████████████████████████████| 1.2 MB 59.6 MB/s 
     |████████████████████████████████| 981 kB 28.7 MB/s 
     |████████████████████████████████| 67 kB 4.9 MB/s 
     |████████████████████████████████| 63 kB 1.4 MB/s 
     |████████████████████████████████| 6.8 MB 36.8 MB/s 
     |████████████████████████████████| 895 kB 42.4 MB/s 
     |████████████████████████████████| 596 kB 42.9 MB/s 
  Created wheel for gdown: filename=gdown-3.12.2-py3-none-any.whl size=9704 sha256=7cc48292609610ef52287b77450924c783524a4b39bfe1

In [4]:
from flair.embeddings import WordEmbeddings
from flair.data import Sentence

# initiate embedding class WordEmbeddings
glove_embedding = WordEmbeddings('de-crawl')

2022-01-28 11:02:21,190 https://flair.informatik.hu-berlin.de/resources/embeddings/token/de-crawl-fasttext-300d-1M.vectors.npy not found in cache, downloading to /tmp/tmpgm9z0pvf


100%|██████████| 1199998928/1199998928 [01:23<00:00, 14376257.97B/s]

2022-01-28 11:03:45,099 copying /tmp/tmpgm9z0pvf to cache at /root/.flair/embeddings/de-crawl-fasttext-300d-1M.vectors.npy


2022-01-28 11:03:49,443 removing temp file /tmp/tmpgm9z0pvf
2022-01-28 11:03:51,246 https://flair.informatik.hu-berlin.de/resources/embeddings/token/de-crawl-fasttext-300d-1M not found in cache, downloading to /tmp/tmpi92ozx53


100%|██████████| 42677439/42677439 [00:02<00:00, 15918538.71B/s]

2022-01-28 11:03:54,338 copying /tmp/tmpi92ozx53 to cache at /root/.flair/embeddings/de-crawl-fasttext-300d-1M


2022-01-28 11:03:54,393 removing temp file /tmp/tmpi92ozx53


In [5]:
# english words embedding
en_embedding = WordEmbeddings('glove')

2022-01-28 11:11:29,152 https://flair.informatik.hu-berlin.de/resources/embeddings/token/glove.gensim.vectors.npy not found in cache, downloading to /tmp/tmpfw9wpj6v


100%|██████████| 160000128/160000128 [00:09<00:00, 17173250.59B/s]

2022-01-28 11:11:38,887 copying /tmp/tmpfw9wpj6v to cache at /root/.flair/embeddings/glove.gensim.vectors.npy


2022-01-28 11:11:39,221 removing temp file /tmp/tmpfw9wpj6v
2022-01-28 11:11:39,990 https://flair.informatik.hu-berlin.de/resources/embeddings/token/glove.gensim not found in cache, downloading to /tmp/tmppolwh2_m


100%|██████████| 21494764/21494764 [00:01<00:00, 11963638.44B/s]

2022-01-28 11:11:42,184 copying /tmp/tmppolwh2_m to cache at /root/.flair/embeddings/glove.gensim
2022-01-28 11:11:42,213 removing temp file /tmp/tmppolwh2_m


In [6]:
def embed(word):
    # create sentence.
    sentence = Sentence(word)

    # embed a sentence using glove.
    glove_embedding.embed(sentence)

    # now check out the embedded tokens.
    for token in sentence:
        #print(token)
        #print(token.embedding)
        torch_tensor = token.embedding
        np_arr = torch_tensor.cpu().detach().numpy()
        return(np_arr)    

In [7]:
def embed_en(word):
    # create sentence.
    sentence = Sentence(word)

    # embed a sentence using glove.
    en_embedding.embed(sentence)

    # now check out the embedded tokens.
    for token in sentence:
        #print(token)
        #print(token.embedding)
        torch_tensor = token.embedding
        np_arr = torch_tensor.cpu().detach().numpy()
        return(np_arr) 

In [8]:
# To check the method embed with an example
embed('Alles')

array([-0.0312,  0.0246,  0.0563, -0.    ,  0.0175, -0.06  , -0.0356,
       -0.0589,  0.0037,  0.0257, -0.0082, -0.048 ,  0.0091, -0.0106,
       -0.0848, -0.1172,  0.0391, -0.0382, -0.0252, -0.0812,  0.0134,
        0.0119,  0.027 ,  0.015 ,  0.0026,  0.0277,  0.0092,  0.0222,
        0.0159,  0.002 ,  0.0974, -0.0257, -0.0019, -0.0929,  0.0111,
       -0.0111, -0.0229,  0.1161, -0.0345, -0.1042,  0.0099, -0.054 ,
        0.0373, -0.0115, -0.0523, -0.0097,  0.0188,  0.0204,  0.0048,
       -0.0762, -0.0294, -0.0072, -0.0143, -0.0305, -0.0112,  0.0083,
       -0.0064,  0.0513, -0.0103, -0.102 , -0.0358, -0.0458, -0.0349,
        0.0992,  0.0205, -0.0247,  0.0157, -0.0332,  0.009 , -0.1691,
       -0.0371, -0.0963, -0.0576,  0.0081, -0.0069,  0.0516,  0.041 ,
        0.0151,  0.0144, -0.0216,  0.0075,  0.0487, -0.0406, -0.0249,
       -0.05  ,  0.1426, -0.0039, -0.0336,  0.0233, -0.0105,  0.0521,
       -0.0111, -0.0382,  0.0133,  0.0094,  0.0949, -0.0411, -0.0459,
        0.0366, -0.0

In [10]:
pip install nltk

In [11]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [12]:
#gloveFile = "glove.6B.50d.txt"
import numpy as np
def loadGloveModel(gloveFile):
    print ("Loading Glove Model")
    with open(gloveFile, encoding="utf8" ) as f:
        content = f.readlines()
    model = {}
    for line in content:
        splitLine = line.split()
        word = splitLine[0]
        embedding = np.array([float(val) for val in splitLine[1:]])
        model[word] = embedding
    print ("Done.",len(model)," words loaded!")
    return model

import re
from nltk.corpus import stopwords
import pandas as pd


def preprocess(raw_text):

    # keep only words
    letters_only_text = re.sub("[^a-zA-Z]", " ", raw_text)

    # convert to lower case and split 
    words = letters_only_text.lower().split()

    # remove stopwords
    stopword_set = set(stopwords.words("german"))
    cleaned_words = list(set([w for w in words if w not in stopword_set]))

    return cleaned_words

def cosine_distance_between_two_words(word1, word2):
    import scipy
    return (1- scipy.spatial.distance.cosine(embed(word1), embed(word2)))

def calculate_heat_matrix_for_two_sentences(s1,s2):
    s1 = preprocess(s1)
    s2 = preprocess(s2)
    result_list = [[cosine_distance_between_two_words(word1, word2) for word2 in s2] for word1 in s1]
    result_df = pd.DataFrame(result_list)
    result_df.columns = s2
    result_df.index = s1
    return result_df

def cosine_distance_wordembedding_method(s1, s2):
    import scipy
    vector_1 = np.mean([embed(word) for word in preprocess(s1)],axis=0)
    vector_2 = np.mean([embed(word) for word in preprocess(s2)],axis=0)
    cosine = scipy.spatial.distance.cosine(vector_1, vector_2)
    return(round((1-cosine)*100,2))

import seaborn as sns
import matplotlib.pyplot as plt    

def heat_map_matrix_between_two_sentences(s1,s2):
    df = calculate_heat_matrix_for_two_sentences(s1,s2)
    #fig, ax = plt.subplots(figsize=(5,5)) 
    #ax_blue = sns.heatmap(df, cmap="YlGnBu")
    #pyplot.show()
    #ax_red = sns.heatmap(df)
    #print()
    return cosine_distance_wordembedding_method(s1, s2)
#
ss1 = 'ai artificial Intelligence maschinelles Lernen künstliche Intelligenz ki'
ss2 = 'IBM und Uni Stuttgart kooperieren um die KI-Forschung in Deutschland voranzutreiben! #ibm4ins #KI #IBMWatson'
#model = loadGloveModel(gloveFile)
heat_map_matrix_between_two_sentences(ss1,ss2)

/usr/local/lib/python3.7/dist-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in float_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


75.24

In [13]:
#gloveFile = "glove.6B.50d.txt"
import numpy as np
import re
from nltk.corpus import stopwords
import pandas as pd


def preprocess_en(raw_text):

    # keep only words
    letters_only_text = re.sub("[^a-zA-Z]", " ", raw_text)

    # convert to lower case and split 
    words = letters_only_text.lower().split()

    # remove stopwords
    stopword_set = set(stopwords.words("english"))
    cleaned_words = list(set([w for w in words if w not in stopword_set]))

    return cleaned_words

def cosine_distance_between_two_words_en(word1, word2):
    import scipy
    return (1- scipy.spatial.distance.cosine(embed_en(word1), embed_en(word2)))

def calculate_heat_matrix_for_two_sentences_en(s1,s2):
    s1 = preprocess_en(s1)
    s2 = preprocess_en(s2)
    result_list = [[cosine_distance_between_two_words_en(word1, word2) for word2 in s2] for word1 in s1]
    result_df = pd.DataFrame(result_list)
    result_df.columns = s2
    result_df.index = s1
    return result_df

def cosine_distance_wordembedding_method_en(s1, s2):
    import scipy
    vector_1 = np.mean([embed_en(word) for word in preprocess_en(s1)],axis=0)
    vector_2 = np.mean([embed_en(word) for word in preprocess_en(s2)],axis=0)
    cosine = scipy.spatial.distance.cosine(vector_1, vector_2)
    return(round((1-cosine)*100,2))

import seaborn as sns
import matplotlib.pyplot as plt

def heat_map_matrix_between_two_sentences_en(s1,s2):
    df = calculate_heat_matrix_for_two_sentences_en(s1,s2)
    #fig, ax = plt.subplots(figsize=(5,5)) 
    #ax_blue = sns.heatmap(df, cmap="YlGnBu")
    # ax_red = sns.heatmap(df)
    return cosine_distance_wordembedding_method_en(s1, s2)

ss1 = 'ai artificial Intelligence machine learning'
ss2 = 'How well do you understand the information &amp; knowledge within your organization?  Are you ready to leverage the benefits of #AI for you stakeholders? https://t.co/wpclrx0nDQ'

#model = loadGloveModel(gloveFile)
heat_map_matrix_between_two_sentences_en(ss1,ss2)

/usr/local/lib/python3.7/dist-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in float_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


73.76

In [15]:
#importing the ai dataset

import pandas as pd
df = pd.read_csv('ai_tweets_date.csv')
df_de = df[df['lang']=='de']
df_en = df[df['lang']=='en']

print(df_de.shape)
print(df_en.shape)
print(df_de.head())
print(df_en.head())

(20929, 4)
(1117, 4)
   Unnamed: 0  ... lang
0           0  ...   de
1           1  ...   de
2           2  ...   de
3           3  ...   de
7           7  ...   de

[5 rows x 4 columns]
     Unnamed: 0  ... lang
4             4  ...   en
5             5  ...   en
6             6  ...   en
143         143  ...   en
247         247  ...   en

[5 rows x 4 columns]


In [16]:
df_de.drop('Unnamed: 0', inplace = True, axis = 1)
df_en.drop('Unnamed: 0', inplace = True, axis = 1)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [18]:
de_tweets = df_de['all_text'].to_numpy()
en_tweets = df_en['all_text'].to_numpy()
de_date = df_de['created_at_string'].to_numpy()
en_date = df_en['created_at_string'].to_numpy()

In [ ]:
#calc. the relevance score for each de tweet and storing it in de_dist

de_list = []
ss1 = 'ai artificial Intelligence maschinelles Lernen künstliche Intelligenz ki'
for i in range(len(de_tweets)):
    ss2 = de_tweets[i]
    x = heat_map_matrix_between_two_sentences(ss1,ss2)
    de_list.append(x)
    
de_relevance = np.array(de_list)

/usr/local/lib/python3.7/dist-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in float_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


In [ ]:
#calc. the relevance score for each eng tweet and storing it in en_dist

en_list = []
ss1 = 'ai artificial Intelligence machine learning'
for i in range(len(en_tweets)):
    ss2 = en_tweets[i]
    x = heat_map_matrix_between_two_sentences_en(ss1,ss2)
    en_list.append(x)
    
en_relevance = np.array(en_list)

In [ ]:
de_result = pd.DataFrame(list(zip(de_date,de_tweets,de_relevance)), columns = ['date','all_text','relevance'])
en_result = pd.DataFrame(list(zip(en_date,en_tweets,en_relevance)), columns = ['date','all_text','relevance'])

In [ ]:
frames = [de_result, en_result]

df_comb = pd.concat(frames)

In [ ]:
df_comb = df_comb.sort_values(by="date")

In [ ]:
df_comb.drop('all_text', inplace = True, axis = 1)

In [ ]:
df_comb = df_comb.groupby('date').mean()

In [ ]:
df_comb.to_csv('ai_relevance.csv')


Refernce : [link text](https://github.com/flairNLP/flair/blob/master/resources/docs/TUTORIAL_3_WORD_EMBEDDING.md)

